In [1]:
from Modified_lux3_wrapper.modified_wrappers_20250228_01 import ModifiedLuxAIS3GymEnv
import numpy as np
from Modified_stablebaseline3_PPO.modified_ppo_20250228_01 import PPO
import torch
from torch.optim import AdamW
import os
import copy
from GreedyLRScheduler import GreedyLR
from luxai_s3.wrappers import LuxAIS3GymEnv

In [2]:
torch.set_float32_matmul_precision('medium')
torch._dynamo.config.capture_scalar_outputs = True
torch._dynamo.config.cache_size_limit = 128
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
np.set_printoptions(linewidth=200)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:512"
torch.backends.cudnn.benchmark = True
torch.cuda.set_per_process_memory_fraction(0.8)
torch.cuda.empty_cache()

In [3]:
init_env = ModifiedLuxAIS3GymEnv(numpy_output=True)

In [4]:
model = PPO("MultiInputPolicy", init_env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
model.policy

MultiInputActorCriticPolicy(
  (features_extractor): CombinedExtractor(
    (extractors): ModuleDict(
      (enemy_energies): Flatten(start_dim=1, end_dim=-1)
      (enemy_positions): Flatten(start_dim=1, end_dim=-1)
      (enemy_spawn_location): Flatten(start_dim=1, end_dim=-1)
      (enemy_visible_mask): Flatten(start_dim=1, end_dim=-1)
      (map_explored_status): Flatten(start_dim=1, end_dim=-1)
      (map_features_energy): Flatten(start_dim=1, end_dim=-1)
      (map_features_tile_type): Flatten(start_dim=1, end_dim=-1)
      (match_steps): Flatten(start_dim=1, end_dim=-1)
      (my_spawn_location): Flatten(start_dim=1, end_dim=-1)
      (relic_nodes): Flatten(start_dim=1, end_dim=-1)
      (relic_nodes_mask): Flatten(start_dim=1, end_dim=-1)
      (sensor_mask): Flatten(start_dim=1, end_dim=-1)
      (steps): Flatten(start_dim=1, end_dim=-1)
      (team_id): Flatten(start_dim=1, end_dim=-1)
      (team_points): Flatten(start_dim=1, end_dim=-1)
      (team_wins): Flatten(start_dim=

In [6]:
model.policy.device

device(type='cuda', index=0)

In [7]:
obs = {
    "enemy_energies": np.random.randint(-800, 401, size=(1, 16,), dtype=np.int32),
    "enemy_positions": np.random.randint(-1, 24, size=(1, 16, 2), dtype=np.int32),
    "enemy_spawn_location": np.random.randint(-1, 24, size=(1, 2,), dtype=np.int32),
    "enemy_visible_mask": np.random.randint(0, 2, size=(1, 16,), dtype=np.int32),
    "map_explored_status": np.random.randint(0, 2, size=(1, 24, 24), dtype=np.int32),
    "map_features_energy": np.random.randint(-7, 10, size=(1, 24, 24), dtype=np.int32),
    "map_features_tile_type": np.random.randint(-1, 3, size=(1, 24, 24), dtype=np.int32),
    "match_steps": np.random.randint(0, 101, size=(1, 1,), dtype=np.int32),
    "my_spawn_location": np.random.randint(-1, 24, size=(1, 2,), dtype=np.int32),
    "relic_nodes": np.random.randint(-1, 24, size=(1, 6, 2), dtype=np.int32),
    "relic_nodes_mask": np.random.randint(0, 2, size=(1, 6,), dtype=np.int32),
    "sensor_mask": np.random.randint(0, 2, size=(1, 24, 24), dtype=np.int32),
    "steps": np.random.randint(0, 506, size=(1, 1,), dtype=np.int32),
    "team_id": np.random.randint(0, 2, size=(1, 1,), dtype=np.int32),
    "team_points": np.random.randint(0, 2501, size=(1, 2,), dtype=np.int32),
    "team_wins": np.random.randint(0, 4, size=(1, 2,), dtype=np.int32),
    "unit_active_mask": np.random.randint(0, 2, size=(1, 16,), dtype=np.int32),
    "unit_energies": np.random.randint(-800, 401, size=(1, 16,), dtype=np.int32),
    "unit_move_cost": np.random.randint(1, 6, size=(1, 1, ), dtype=np.int32),
    "unit_positions": np.random.randint(-1, 24, size=(1, 16, 2), dtype=np.int32),
    "unit_sap_cost": np.random.randint(30, 51, size=(1, 1, ), dtype=np.int32),
    "unit_sap_range": np.random.randint(3, 8, size=(1, 1, ), dtype=np.int32),
    "unit_sensor_range": np.random.randint(2, 5, size=(1, 1, ), dtype=np.int32),
}

obs = {k: torch.tensor(v, dtype=torch.float32, device="cuda") for k, v in obs.items()}

with torch.no_grad():
    action_distribution, value, log = model.policy.forward(obs)

actions = copy.deepcopy(action_distribution.reshape(-1, 16, 3))
actions[:, :, 1] = actions[:, :, 1] - 7
actions[:, :, 2] = actions[:, :, 2] - 7
actions

tensor([[[ 5,  1,  5],
         [ 0,  0, -5],
         [ 5,  7, -6],
         [ 2,  2, -2],
         [ 2, -6, -3],
         [ 3, -7,  5],
         [ 5,  0, -1],
         [ 5,  0, -7],
         [ 5, -5,  4],
         [ 3,  1, -5],
         [ 4, -6,  5],
         [ 2,  7,  2],
         [ 2,  2, -6],
         [ 5,  3,  7],
         [ 5,  3, -4],
         [ 2,  0,  0]]], device='cuda:0')

In [8]:
actions.shape

torch.Size([1, 16, 3])

In [9]:
actions.squeeze().shape

torch.Size([16, 3])

In [10]:
for i, v in enumerate(actions.squeeze()):
    print(i, v)

0 tensor([5, 1, 5], device='cuda:0')
1 tensor([ 0,  0, -5], device='cuda:0')
2 tensor([ 5,  7, -6], device='cuda:0')
3 tensor([ 2,  2, -2], device='cuda:0')
4 tensor([ 2, -6, -3], device='cuda:0')
5 tensor([ 3, -7,  5], device='cuda:0')
6 tensor([ 5,  0, -1], device='cuda:0')
7 tensor([ 5,  0, -7], device='cuda:0')
8 tensor([ 5, -5,  4], device='cuda:0')
9 tensor([ 3,  1, -5], device='cuda:0')
10 tensor([ 4, -6,  5], device='cuda:0')
11 tensor([2, 7, 2], device='cuda:0')
12 tensor([ 2,  2, -6], device='cuda:0')
13 tensor([5, 3, 7], device='cuda:0')
14 tensor([ 5,  3, -4], device='cuda:0')
15 tensor([2, 0, 0], device='cuda:0')


In [11]:
action_distribution.shape

torch.Size([1, 48])

In [12]:
action_distribution

tensor([[ 5,  8, 12,  0,  7,  2,  5, 14,  1,  2,  9,  5,  2,  1,  4,  3,  0, 12,
          5,  7,  6,  5,  7,  0,  5,  2, 11,  3,  8,  2,  4,  1, 12,  2, 14,  9,
          2,  9,  1,  5, 10, 14,  5, 10,  3,  2,  7,  7]], device='cuda:0')

In [13]:
value.shape

torch.Size([1, 1])

In [14]:
value

tensor([[0.3088]], device='cuda:0')

In [15]:
log.shape

torch.Size([1])

In [16]:
log

tensor([-115.3207], device='cuda:0')

In [17]:
model.policy.evaluate_actions(obs, action_distribution)

(tensor([[0.3088]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([-115.3207], device='cuda:0', grad_fn=<SumBackward1>),
 tensor([115.3256], device='cuda:0', grad_fn=<SumBackward1>))

In [21]:
value + -log

tensor([[115.6296]], device='cuda:0')

In [ ]:
with torch.no_grad():
    action_distribution2, value2, log2 = model.policy(obs)

In [ ]:
action_distribution2

In [ ]:
value2

In [ ]:
log2

In [ ]:
with torch.no_grad():
    something = model.policy(obs)

In [ ]:
len(something)

In [ ]:
something

In [22]:
features = model.policy.extract_features(obs)

In [23]:
features.shape

torch.Size([1, 2466])

In [24]:
features

tensor([[  97., -104.,  359.,  ...,   33.,    3.,    4.]], device='cuda:0')

In [25]:
latent_pi, latent_vf = model.policy.mlp_extractor(features)

In [26]:
latent_pi.shape

torch.Size([1, 64])

In [27]:
latent_pi

tensor([[-0.2595, -0.0433,  0.9991,  0.9792,  0.4206, -0.2526, -0.6704, -0.5540,
         -0.0934, -0.9404, -0.6263,  0.8389,  0.9515, -0.8615, -0.8529, -0.4819,
          0.5591,  0.8549, -0.4783,  0.0616, -0.1377, -0.0559, -0.2873,  0.9998,
         -0.9815, -0.1951, -0.8463,  0.8768,  0.9118,  0.5954,  0.7028, -0.6369,
          0.6047,  0.7684, -0.3009, -0.7373,  0.0529, -0.3780,  0.4561,  0.8869,
         -0.9491,  0.4357,  0.9860, -0.7408,  0.6284,  0.7749,  0.5563,  0.9782,
         -0.5009, -0.9394,  0.4296, -0.6890,  0.9836,  0.9156, -0.8435,  0.4221,
         -0.2909,  0.0390, -0.0872,  0.4957,  0.9817,  0.9930,  0.8953, -0.8103]],
       device='cuda:0', grad_fn=<TanhBackward0>)

In [28]:
latent_vf.shape

torch.Size([1, 64])

In [29]:
latent_vf

tensor([[-0.5122, -0.9038, -0.9311,  0.5610, -0.6714, -0.2262,  0.7510, -0.1469,
          0.8442,  0.9737, -0.9787, -0.9602,  0.9222,  0.7987,  0.8580, -0.1741,
         -0.8663, -0.8598,  0.9726,  0.9065,  0.8920,  0.4981,  0.1220,  0.9883,
          0.3116, -0.3982,  0.2209, -0.6473,  0.9317, -0.9827, -0.1555, -0.9034,
          0.9014,  0.8704, -0.6282, -0.3511,  0.5608,  0.8297, -0.4596,  0.4936,
         -0.7391, -0.7619,  0.9309, -0.7326,  0.9359,  0.8413, -0.6815, -0.7928,
         -0.2650, -0.3836, -0.9805,  0.6087,  0.7684,  0.7558,  0.9632, -0.2257,
         -0.4769, -0.3067,  0.5773,  0.7750,  0.9995,  0.9967, -0.9689,  0.9716]],
       device='cuda:0', grad_fn=<TanhBackward0>)

In [30]:
distribution = model.policy._get_action_dist_from_latent(latent_pi)
distribution

In [31]:
distribution.get_actions().shape

torch.Size([1, 48])

In [40]:
distribution.get_actions()

tensor([[ 2,  5,  6,  5,  5,  5,  2,  3,  2,  5,  7,  0,  2,  2,  6,  3,  2,  1,
          1, 14, 14,  3,  0, 13,  0, 11,  1,  5,  3,  7,  0, 13,  8,  0,  5,  4,
          5,  3,  3,  5, 14,  1,  5, 12,  3,  4, 12,  9]], device='cuda:0')

In [32]:
log_prob = distribution.log_prob(action_distribution)

In [33]:
log_prob.shape

torch.Size([1])

In [34]:
log_prob

tensor([-115.3207], device='cuda:0', grad_fn=<SumBackward1>)

In [35]:
values = model.policy.value_net(latent_vf)
values.shape

torch.Size([1, 1])

In [36]:
values

tensor([[0.3088]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [37]:
entropy = distribution.entropy()

In [38]:
entropy.shape

torch.Size([1])

In [39]:
entropy

tensor([115.3256], device='cuda:0', grad_fn=<SumBackward1>)

In [ ]:
from stable_baselines3.common.buffers import DictRolloutBuffer

In [ ]:
short_term_rollout_buffer = DictRolloutBuffer(1, env.observation_space, env.action_space, device="cuda")
short_term_rollout_buffer

In [ ]:
short_term_rollout_buffer.add(obs, action_distribution, values, log_prob, entropy)

In [ ]:
for rollout_data in short_term_rollout_buffer.get(1):
    print(rollout_data)

In [ ]:
def point_gain_reward_func(reward_score) -> float:

    return reward_score * 20 if reward_score > 0.0 else -1

def match_won_reward_func(match_won) -> float:

    return 5000.0 if match_won else 0.0

def match_lost_reward_func(match_lost) -> float:

    return -3000.0 if match_lost else 0.0

def game_won_reward_func(game_won) -> float:

    return 1000000.0 if game_won else 0.0

def game_lost_reward_func(game_lost) -> float:

    return -1000000.0 if game_lost else 0.0

def map_reveal_reward_func(map_reveal_score):

    return map_reveal_score * 10

def attack_reward_func(actions, sap_range, enemy_unit_mask) -> float:

    attack_score = 0.0
    
    for i, action in enumerate(actions):
        action_num, dx, dy = action[0], action[1], action[2]
        if action_num >= 5:
            if enemy_unit_mask.sum() != 0:
                sap_action_range = max(abs(dx), abs(dy))
                if sap_action_range > sap_range:
                    attack_score -= 0.5
            else:
                attack_score -= 5.0
    
    return attack_score

def next_position_calculator(action_num, unit_positions):
    # 0: stay, 1: up, 2: right, 3: down, 4: left

    if action_num == 1:
        next_position = (unit_positions[0], unit_positions[1] - 1)
    elif action_num == 2:
        next_position = (unit_positions[0] + 1, unit_positions[1])
    elif action_num == 3:
        next_position = (unit_positions[0], unit_positions[1] + 1)
    elif action_num == 4:
        next_position = (unit_positions[0] - 1, unit_positions[1])
    else:
        next_position = unit_positions
    
    return next_position

def movement_reward_func(actions, obs, team_id) -> float:

    movement_score = 0.0

    for i, action in enumerate(actions):
        action_num, dx, dy = action[0], action[1], action[2]
        unit_positions = obs["units"]["position"][team_id][i]
        unit_energy = obs["units"]["energy"][team_id][i]

        # give penalty if try to move unit that doesn't exist
        if (unit_positions == (-1, -1)).sum() == 2 and action_num != 0:
            movement_score -= 0.25
        
        # give penalty if dx or dy is not 0 when not attacking
        if action_num != 5:
            if dx != 0 or dy != 0:
                movement_score -= 0.25

        
        if unit_positions[0] >= 0 and unit_positions[1] >= 0:
            # give penalty if try to move unit that has no energy
            if unit_energy <= 0 and action_num != 0:
                movement_score -= 0.25
        
        # give penalty if try to move unit out of map
        next_position = next_position_calculator(action_num, unit_positions)
        if next_position[0] < 0 or next_position[1] < 0 or next_position[0] > 23 or next_position[1] > 23:
            movement_score -= 0.5
        else:
            movement_score += 2.0
    

    return movement_score

def relic_discovery_reward_func(relic_discovery_reward) -> float:

    return relic_discovery_reward * 100

In [ ]:
obs_all["player_0"]["team_points"]

In [ ]:
obs_all["player_0"].keys()

In [ ]:
obs_all["player_0"]["units_mask"]

In [ ]:
env_cfg = info["params"]
env_cfg

In [ ]:
class TrainPPO:
    def __init__(
        self,
        model_0,
        model_1,
        reward_functions: list,
        num_games=1000,
        learning_rate=5e-4,
        weight_decay=0.01,
    ):
        self.model_0 = model_0
        self.model_1 = model_1
        self.reward_functions = reward_functions
        self.num_games = num_games
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

        self.optimizer_0 = AdamW(self.model_0.policy.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay, fused=True)
        self.optimizer_1 = AdamW(self.model_1.policy.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay, fused=True)

        self.scheduler_0 = GreedyLR(self.optimizer_0, cooldown=3, min_lr=1e-7, max_lr=5e-4)
        self.scheduler_1 = GreedyLR(self.optimizer_1, cooldown=3, min_lr=1e-7, max_lr=5e-4)

        self.env = LuxAIS3GymEnv(numpy_output=True)

    def train(self):

        for game in range(1, self.num_games + 1):
            print("="*15 + f"Game {game} Started" + "="*15)

            obs_all, info = self.env.reset()
            self.env_cfg = info['params']

            game_ended = False

            player_0_previous_score = 0.0
            player_1_previous_score = 0.0

            first_spawn = False

            self.spawn_location = np.array([[-1, -1], [-1, -1]], dtype=np.int32)

            self.map_explored_status = np.zeros((2, 24, 24), dtype=bool)

            player_0_previous_map_explored_status_score = self.map_explored_status[0].sum()
            player_1_previous_map_explored_status_score = self.map_explored_status[1].sum()

            player_0_match_won_num = 0
            player_1_match_won_num = 0

            player_0_previous_relic_discovery_points = 0
            player_1_previous_relic_discovery_points = 0

            victor = None

            while game_ended is not True:

                player_0_current_score = obs_all['player_0']['team_points'][0]
                player_1_current_score = obs_all['player_1']['team_points'][1]

                player_0_reward_score = player_0_current_score - player_0_previous_score
                player_1_reward_score = player_1_current_score - player_1_previous_score

                player_0_previous_score = player_0_current_score
                player_1_previous_score = player_1_current_score

                current_match_step = obs_all["player_0"]["match_steps"]

                player_0_match_won = False
                player_0_match_lost = False
                player_1_match_won = False
                player_1_match_lost = False

                player_0_game_won = False
                player_0_game_lost = False
                player_1_game_won = False
                player_1_game_lost = False

                if current_match_step == 100:
                    if player_0_current_score > player_1_current_score:
                        player_0_match_won = True
                        player_1_match_lost = True
                        player_0_match_won_num += 1
                    elif player_0_current_score < player_1_current_score:
                        player_0_match_lost = True
                        player_1_match_won = True
                        player_1_match_won_num += 1

                if player_0_match_won_num >= 3:
                    game_ended = True
                    print("Player 0 won the game.")
                    victor = "player_0"
                    break

                if player_1_match_won_num >= 3:
                    game_ended = True
                    print("Player 1 won the game.")
                    victor = "player_1"
                    break

                player_0_unit_positions = np.array(obs_all['player_0']["units"]["position"][0])
                player_1_unit_positions = np.array(obs_all['player_1']["units"]["position"][1])

                player_0_unit_mask = np.array(obs_all['player_0']["units_mask"][0])
                player_1_unit_mask = np.array(obs_all['player_1']["units_mask"][1])

                player_0_available_unit_ids = np.where(player_0_unit_mask)[0]
                player_1_available_unit_ids = np.where(player_1_unit_mask)[0]

                if player_0_available_unit_ids.shape[0] == 0:
                    pass
                else:
                    if first_spawn == False:
                        player_0_first_unit_id = player_0_available_unit_ids[0]
                        player_0_first_unit_pos = player_0_unit_positions[player_0_first_unit_id]
                        self.spawn_location[0] = (player_0_first_unit_pos[0], player_0_first_unit_pos[1])
                        player_1_first_unit_id = player_1_available_unit_ids[0]
                        player_1_first_unit_pos = player_1_unit_positions[player_1_first_unit_id]
                        self.spawn_location[1] = (player_1_first_unit_pos[0], player_1_first_unit_pos[1])
                        first_spawn = True

                player_0_map_features = obs_all['player_0']['map_features']
                player_1_map_features = obs_all['player_1']['map_features']

                player_0_current_map_tile_type = player_0_map_features['tile_type'].T
                player_1_current_map_tile_type = player_1_map_features['tile_type'].T

                self.map_explored_status[0][player_0_current_map_tile_type != -1] = True
                self.map_explored_status[1][player_1_current_map_tile_type != -1] = True

                player_0_current_map_explored_status_score = self.map_explored_status[0].sum()
                player_1_current_map_explored_status_score = self.map_explored_status[1].sum()

                player_0_map_explored_status_reward = player_0_current_map_explored_status_score - player_0_previous_map_explored_status_score
                player_1_map_explored_status_reward = player_1_current_map_explored_status_score - player_1_previous_map_explored_status_score

                player_0_previous_map_explored_status_score = player_0_current_map_explored_status_score
                player_1_previous_map_explored_status_score = player_1_current_map_explored_status_score

                ### Reward caclulation
                player_0_relic_point_reward = point_gain_reward_func(player_0_reward_score)
                player_1_relic_point_reward = point_gain_reward_func(player_1_reward_score)

                player_0_match_won_reward = match_won_reward_func(player_0_match_won)
                player_0_match_lost_reward = match_lost_reward_func(player_0_match_lost)
                player_1_match_won_reward = match_won_reward_func(player_1_match_won)
                player_1_match_lost_reward = match_lost_reward_func(player_1_match_lost)

                player_0_game_won_reward = game_won_reward_func(player_0_game_won)
                player_0_game_lost_reward = game_lost_reward_func(player_0_game_lost)
                player_1_game_won_reward = game_won_reward_func(player_1_game_won)
                player_1_game_lost_reward = game_lost_reward_func(player_1_game_lost)

                player_0_map_reveal_reward = map_reveal_reward_func(player_0_map_explored_status_reward)
                player_1_map_reveal_reward = map_reveal_reward_func(player_1_map_explored_status_reward)



                ### model input
                player_0_model_input = self.prepare_model_input(obs_all["player_0"], 0)
                player_1_model_input = self.prepare_model_input(obs_all["player_1"], 1)

                with torch.no_grad():
                    player_0_action_distribution, player_0_value, player_0_log = self.model_0.forward(player_0_model_input)
                    player_1_action_distribution, player_1_value, player_1_log = self.model_1.forward(player_1_model_input)

                player_0_actions = copy.deepcopy(player_0_action_distribution.reshape(-1, 16, 3)).squeeze()
                player_0_actions[:, 1] = player_0_actions[:, 1] - 7
                player_0_actions[:, 2] = player_0_actions[:, 2] - 7
                player_1_actions = copy.deepcopy(player_1_action_distribution.reshape(-1, 16, 3)).squeeze()
                player_1_actions[:, 1] = player_1_actions[:, 1] - 7
                player_1_actions[:, 2] = player_1_actions[:, 2] - 7

                player_0_attack_reward = attack_reward_func(player_0_actions, self.env_cfg["unit_sap_range"], player_1_unit_mask)
                player_1_attack_reward = attack_reward_func(player_1_actions, self.env_cfg["unit_sap_range"], player_0_unit_mask)

                player_0_movement_reward = movement_reward_func(player_0_actions, obs_all["player_0"], 0)
                player_1_movement_reward = movement_reward_func(player_1_actions, obs_all["player_1"], 1)

                player_0_reward = player_0_relic_point_reward + player_0_match_won_reward + player_0_match_lost_reward + player_0_game_won_reward + player_0_game_lost_reward + player_0_map_reveal_reward + player_0_attack_reward + player_0_movement_reward
                player_1_reward = player_1_relic_point_reward + player_1_match_won_reward + player_1_match_lost_reward + player_1_game_won_reward + player_1_game_lost_reward + player_1_map_reveal_reward + player_1_attack_reward + player_1_movement_reward

                player_0_features = self.model_0.extract_features(player_0_model_input)
                player_1_features = self.model_1.extract_features(player_1_model_input)

                player_0_latent_pi, player_0_latent_vf = self.model_0.mlp_extractor(player_0_features)
                player_1_latent_pi, player_1_latent_vf = self.model_1.mlp_extractor(player_1_features)

                player_0_distribution = self.model_0._get_action_dist_from_latent(player_0_latent_pi)
                player_1_distribution = self.model_1._get_action_dist_from_latent(player_1_latent_pi)

                player_0_log_prob = player_0_distribution.log_prob(player_0_action_distribution)
                player_1_log_prob = player_1_distribution.log_prob(player_1_action_distribution)

                player_0_values = self.model_0.value_net(player_0_latent_vf)
                player_1_values = self.model_1.value_net(player_1_latent_vf)

                player_0_entropy = player_0_distribution.entropy()
                player_1_entropy = player_1_distribution.entropy()



        return


    def prepare_model_input(self, obs, my_team_id):
        enemy_team_id = 1 - my_team_id

        model_input = {
            "enemy_energies": obs["units"]["energy"][enemy_team_id],
            "enemy_positions": obs["units"]["position"][enemy_team_id],
            "enemy_spawn_location": self.spawn_location[enemy_team_id],
            "enemy_visible_mask": obs["units_mask"][enemy_team_id],
            "map_explored_status": self.map_explored_status[my_team_id],
            "map_features_energy": obs["map_features"]["energy"],
            "map_features_tile_type": obs["map_features"]["tile_type"],
            "match_steps": np.array([obs["match_steps"]]),
            "my_spawn_location": self.spawn_location[my_team_id],
            "relic_nodes": obs["relic_nodes"],
            "relic_nodes_mask": obs["relic_nodes_mask"],
            "sensor_mask": obs["sensor_mask"],
            "steps": np.array([obs["steps"]]),
            "team_id": np.array([my_team_id]),
            "team_points": obs["team_points"],
            "team_wins": obs["team_wins"],
            "unit_active_mask": obs["units_mask"][my_team_id],
            "unit_energies": obs["units"]["energy"][my_team_id],
            "unit_move_cost": np.array([self.env_cfg["unit_move_cost"]]),
            "unit_positions": obs["units"]["position"][my_team_id],
            "unit_sap_cost": np.array([self.env_cfg["unit_sap_cost"]]),
            "unit_sap_range": np.array([self.env_cfg["unit_sap_range"]]),
            "unit_sensor_range": np.array([self.env_cfg["unit_sensor_range"]]),
        }

        model_input = {k: torch.tensor(np.expand_dims(v, axis=0), dtype=torch.int32, device="cuda") for k, v in model_input.items()}

        return model_input



        


In [ ]:
model.policy.parameters()

In [ ]:
obs_tensor = model.policy.extract_features(obs)
obs_tensor

In [ ]:
obs_tensor.shape

In [ ]:
obs = {
    "enemy_energies": np.random.randint(-800, 401, size=(1, 16,), dtype=np.int32),
    "enemy_positions": np.random.randint(-1, 24, size=(1, 16, 2), dtype=np.int32),
    "enemy_spawn_location": np.random.randint(-1, 24, size=(1, 2,), dtype=np.int32),
    "enemy_visible_mask": np.random.randint(0, 2, size=(1, 16,), dtype=np.int32),
    "map_explored_status": np.random.randint(0, 2, size=(1, 24, 24), dtype=np.int32),
    "map_features_energy": np.random.randint(-7, 10, size=(1, 24, 24), dtype=np.int32),
    "map_features_tile_type": np.random.randint(-1, 3, size=(1, 24, 24), dtype=np.int32),
    "match_steps": np.random.randint(0, 101, size=(1, 1,), dtype=np.int32),
    "my_spawn_location": np.random.randint(-1, 24, size=(1, 2,), dtype=np.int32),
    "relic_nodes": np.random.randint(-1, 24, size=(1, 6, 2), dtype=np.int32),
    "relic_nodes_mask": np.random.randint(0, 2, size=(1, 6,), dtype=np.int32),
    "sensor_mask": np.random.randint(0, 2, size=(1, 24, 24), dtype=np.int32),
    "steps": np.random.randint(0, 506, size=(1, 1,), dtype=np.int32),
    "team_id": np.random.randint(0, 2, size=(1, 1,), dtype=np.int32),
    "team_points": np.random.randint(0, 2501, size=(1, 2,), dtype=np.int32),
    "team_wins": np.random.randint(0, 4, size=(1, 2,), dtype=np.int32),
    "unit_active_mask": np.random.randint(0, 2, size=(1, 16,), dtype=np.int32),
    "unit_energies": np.random.randint(-800, 401, size=(1, 16,), dtype=np.int32),
    "unit_move_cost": np.random.randint(1, 6, size=(1, 1, ), dtype=np.int32),
    "unit_positions": np.random.randint(-1, 24, size=(1, 16, 2), dtype=np.int32),
    "unit_sap_cost": np.random.randint(30, 51, size=(1, 1, ), dtype=np.int32),
    "unit_sap_range": np.random.randint(3, 8, size=(1, 1, ), dtype=np.int32),
    "unit_sensor_range": np.random.randint(2, 5, size=(1, 1, ), dtype=np.int32),
}

In [ ]:
obs = {k: torch.tensor(v, dtype=torch.float32, device="cuda") for k, v in obs.items()}

# Convert observation to tensor and check shape
obs_tensor = model.policy.extract_features(obs)
print(f"Extracted Feature Shape: {obs_tensor.shape}")  # Expected: (batch_size, 2464)

In [ ]:
with torch.no_grad():
    action_distribution, value, log = model.policy.forward(obs)

In [ ]:
action_distribution.shape

In [ ]:
action_distribution

In [ ]:
value.shape

In [ ]:
value

In [ ]:
log.shape

In [ ]:
log

In [ ]:
actions = action_distribution.reshape(-1, 16, 3)
actions

In [ ]:
actions.shape

In [ ]:
model.policy.mlp_extractor = torch.compile(model.policy.mlp_extractor)

In [ ]:
with torch.no_grad():
    action_distribution, value, log = model.policy.forward(obs)

In [ ]:
actions = action_distribution.reshape(-1, 16, 3)
actions

In [ ]:
%pip install --upgrade luxai-s3